In [4]:
import os
import sys

if 'has_changed_dir' not in globals():
    repo_path = os.path.abspath(os.path.join('..'))
    
    if repo_path not in sys.path:
        sys.path.append(repo_path)
    
    os.chdir(repo_path)
    
    globals()['has_changed_dir'] = True
    print(repo_path)
    print(os.getcwd())


# from typing import Optional, Union

# import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from src.train_model import LightningModel




### Load model using ckpt (you dont need it if you convert your model to transformers)

In [5]:
# model_path: str = "models/re-init/"
# tokenizer_path: str = "models/re-init/"
# tokenizer_vocab_file_path: str = "models/re-init/sentencepiece.bpe.model"
# ckpt_path: Optional[Union[str, None]] = "models/it_1/epoch=17-val_loss=1.48184.ckpt"

# model = AutoModelForSeq2SeqLM.from_pretrained(model_path, local_files_only=True)
# model.eval()
# tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, vocab_file = tokenizer_vocab_file_path)


# # Option 1:
# # lightning_model = LightningModel.load_from_checkpoint(ckpt_path, map_location=torch.device("cuda:1"), model = model, tokenizer = tokenizer)

# # Option 2:
# ckpt = torch.load(ckpt_path, map_location=torch.device("cuda:1"))
# lightning_model = LightningModel(model, tokenizer)
# lightning_model.load_state_dict(ckpt['state_dict'])


### Load converted model

In [6]:
""" Load local converted model"""

# final_model_path = "models/final"

# model = AutoModelForSeq2SeqLM.from_pretrained(final_model_path)
# tokenizer = AutoTokenizer.from_pretrained(final_model_path, vocab_file = final_model_path + "/sentencepiece.bpe.model")
# lightning_model = LightningModel(model, tokenizer)


""" Download and load converted model from hub """

model = AutoModelForSeq2SeqLM.from_pretrained("leks-forever/mt5-base")
tokenizer = AutoTokenizer.from_pretrained("leks-forever/mt5-base")
lightning_model = LightningModel(model, tokenizer)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/raki/raki-projects/t5-tuning/.venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Prefixes:    
`"translate Russian to Lezghian: "` - Ru-Lez    
`"translate Lezghian to Russian: "` - Lez-Ru    

In [7]:
sentence: str = "Римдин аскерар ва гьакӀни чӀехи хахамрини  фарисейри ракъурнавай нуькерар Ягьуд галаз багъдиз атана. Абурув виридав яракьар, чирагъар ва шемгьалар гвай."

translation = lightning_model.predict(sentence, prefix="translate Lezghian to Russian: ")

print(translation)

['Когда римские воины и вожди, а также главные священнослужители и блюстители Закона пришли в Иудею, они дали ему вооружённые оружие, браслеты и серьги.']


In [8]:
sentence: str = "Когда Исо услышал это, Он сказал: – Не здоровым нужен врач, а больным."

translation = lightning_model.predict(sentence, prefix="translate Russian to Lezghian: ")

print(translation)

# Реальный перевод: И гафар ван хьайила, Исади абуруз лагьана: – Жерягь сагъбуруз ваъ, начагъбуруз герек я.

['И гафар ван хьайила, Исади лагьана: - Жерягь сагъбуруз ваъ, начагъбуруз герек я.']


In [9]:


sentences = [
    "Давай поедим пиццу, а потом еще немного погуляем!",
    "Я люблю гулять по парку ранним утром, когда воздух свежий и тишина вокруг.",
    "Новый фильм, который мы смотрели вчера, произвёл на меня сильное впечатление",
    "В следующем году я планирую посетить несколько стран, чтобы познакомиться с их культурой.",
    "После долгого рабочего дня приятно расслабиться с книгой и чашкой чая."
]

for sentence in sentences:
    translation = lightning_model.predict(sentence, prefix="translate Russian to Lezghian: ")

    print(translation)


['Куьне тӀили фу неъ, ахпа са тӀимил кьван ша чун.']
['КӀвализ гьахьайла, рагъ экъечӀдай вахтунда цӀайлапанар авай ва вири юкьвал мичӀи хьанвай.']
['Чна акур чаз акур цӀийи кьилелай зун гзаф хъел атана']
['Пакадин йисуз за а чкадиз абурун халкьдин гъавурда чир хьун патал са бязи уьлкведиз къвез кӀанзава.']
['Гьар са кӀвалахдин йикъан няниз хъфидай югъ алукьнавай кӀвалахни хъваз шадвал ацукь.']


### Note: 
You can use the predict method regardless of the Lightning model

In [10]:
def predict(text, prefix, a=32, b=3, max_input_length=1024, num_beams=1, **kwargs):
    inputs = tokenizer(prefix + text, return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)
    result = model.generate(
        **inputs.to(model.device),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams,
        **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)


In [11]:
sentence: str = "Римдин аскерар ва гьакӀни чӀехи хахамрини  фарисейри ракъурнавай нуькерар Ягьуд галаз багъдиз атана. Абурув виридав яракьар, чирагъар ва шемгьалар гвай."

translation = predict(sentence, prefix="translate Lezghian to Russian: ")

print(translation)

['Когда римские воины и вожди, а также главные священнослужители и блюстители Закона пришли в Иудею, они дали ему вооружённые оружие, браслеты и серьги.']
